<a href="https://colab.research.google.com/github/iambikash378/ML-Stuff/blob/main/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision as tv
import torchvision.datasets as datasets
from torchvision.utils import save_image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from vector_quantize_pytorch import VectorQuantize



ModuleNotFoundError: No module named 'vector_quantize_pytorch'

In [ ]:
# Downloading and initialzing the MNIST Trainin and Testing dataset

transform = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.1307,),(0.3081,))])
mnist_trainset = datasets.MNIST(root='./data', train= True, download = False, transform = transform)
mnist_testset = datasets.MNIST(root='./data', train = False, download = True, transform = transform)
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size = 10, shuffle = True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size = 5, shuffle = False)


In [ ]:
class Autoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder,self).__init__()

    self.encoder = nn.Sequential(
      nn.Conv2d(1,4,kernel_size = 5),
      nn.ReLU(True),
      nn.Conv2d(4, 8, kernel_size =5),
      nn.ReLU(True),
    )

    self.quantizer = VectorQuantize(
       dim = 20,
       codebook_size = 8,
       decay = 0.8,
       commitment_weight = 1
    )

    self.decoder = nn.Sequential(
      nn.Linear(10, 400),
      nn.ReLU(True),
      nn.Linear(400, 4000),
      nn.ReLU(True),
      nn.Unflatten(1, (10, 20 , 20)),
      nn.ConvTranspose2d(10, 10, kernel_size = 5),
      nn.ConvTranspose2d(10,1, kernel_size =5),
      nn.Sigmoid(),
    )

  def forward(self, x):
    enc = self.encoder(x)
    qtz, _ , _ = self.quantizer(enc)
    dec = self.decoder(qtz)
    return dec


In [ ]:
# Defining the Hyperparameters

num_epochs = 1
batch_size = 128
model = Autoencoder().cpu()
distance = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum = 0.5)


In [ ]:
# Training the model
for epoch in range(num_epochs):
  for data in train_loader:
    img, _ = data
    img = Variable(img).cpu()
    output = model(img)
    loss = distance(output, img)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('epoch [{}/{}], loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

torch.save(model.state_dict(),'trained_model.pth')

In [ ]:
model.eval()

# Get some test images
images, _ = next(iter(test_loader))

# Reconstruct images
with torch.no_grad():
    reconstructed_images = model(images)

# Display original and reconstructed images
num_images = 5
plt.figure(figsize=(10, 4))
for i in range(num_images):
    # Original image
    plt.subplot(2, num_images, i + 1)
    plt.imshow(np.transpose(images[i], (1, 2, 0)).squeeze(), cmap='gray')
    plt.title('Original')
    plt.axis('off')

    # Reconstructed image
    plt.subplot(2, num_images, i + 1 + num_images)
    plt.imshow(np.transpose(reconstructed_images[i], (1, 2, 0)).squeeze(), cmap='gray')
    plt.title('Reconstructed')
    plt.axis('off')

plt.show()